In [27]:
import sys
import os

import pandas as pd
import numpy as np
import natsort
import random as rn
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
import pyeeg

from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps

import matplotlib.pyplot as plt

import pycrfsuite
import sklearn_crfsuite

#Sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPRegressor
from sklearn.svm import (SVC, SVR)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import (StratifiedKFold, KFold)

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.manifold import TSNE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

seed=42
np.random.seed(seed)
rn.seed(seed)
tf.set_random_seed(seed)
dir_path = os.getcwd()
# https://en.wikipedia.org/wiki/Neural_oscillation
SAMPLE_PER_SEC = 128
DELTA_WAVE = [1, 4]
THETA_WAVE = [4, 8]
ALPHA_WAVE = [7.5, 12.5]
BETA_WAVE = [13, 30]
TOTAL_ENERGY = [0, 64]
# not used
LOW_GAMMA_WAVE = [30, 70]
HIGH_GAMMA_WAVE = [70, 150]

EPSILON =  0.0002

In [5]:
def get_train_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of 4 second sample. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path, csv_file)
    print(f"Reading {csv_file}")
    with open(csv_file, 'r') as csvfile:
        train_reader = pd.read_csv(csvfile)
        train_reader.drop(labels="Id", axis=1, inplace=True)
        
        
    return train_reader.values

def get_target_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of 4 second sample. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path,csv_file)
    with open(csv_file, 'r') as csvfile:
        label_reader = pd.read_csv(csvfile)
        #print("Labels: ", label_reader['id'])
        y = label_reader['y']
        
    y = np.array(y)
    return y

def get_features_emg(X):
    all_featues = []
    for i in tqdm(range(X.shape[0])):
        features = list()
        # https://ieeexplore.ieee.org/document/7748960
        x_i = X[i,:]
        # Root Mean Square (RMS): RMS of EMG
        features.append(mean_squared_error(x_i, np.zeros(x_i.shape)))
        
        #Integrated Absolute Value (IAV)
        features.append(np.sum(np.abs(x_i)))
        
        # Mean Absolute Value (MAV): MAV feature can be expressed as
        features.append(np.mean(x_i))
        
        # TBD:
        # Modified Mean Absolute Value type 1
        # Modified Mean Absolute Value type 2
        
        # Simple Square Integral (SSI): SSI is calculated as
        features.append(np.sum(x_i ** 2))
        
        # Variance (VAR): VAR is calculated as
        features.append(np.var(x_i))
        
        #The 3rd, 4th and 5th temporal moments
        features.append(np.mean(x_i ** 3))
        features.append(np.mean(x_i ** 4))
        features.append(np.mean(x_i ** 5))
        
        # TBD
        # v-Order 
        
        # Waveform Length
        features.append(np.sum(np.abs(np.diff(x_i))))
        
        # Average Amplitude Change
        features.append(np.mean(np.abs(np.diff(x_i))))
        
        # Difference Absolute Standard Deviation Value
        features.append(np.sqrt(np.mean(np.power(np.diff(x_i), 2))))
        
        # AX BASIC FEATUERS
        features.append(np.std(x_i))
        features.append(np.min(x_i))
        features.append(np.max(x_i))
        features.append(np.sum(np.abs(x_i) > EPSILON))
        
        
        
        all_featues.append(features)
    return np.array(all_featues)

def get_features_eeg(X):
    all_featues = []
    # NOT SURE ABOUT THIS VALUES 
    # LETS DOUBLE CHECK
    K_MAX = 6
    SAMPLE_PER_SEC = 32
    FREQ_BANDS = list(range(16))
    TAU = 16
    # embedding dimension
    DE = 32
    
    for i in tqdm(range(X.shape[0])):
        features = list()
        # http://pyeeg.sourceforge.net/
        x_i = X[i,:]
        
        # Power Spectral Intensity (PSI) and Relative Intensity Ratio (RIR)	bin_power()	Two 1-D vectors
        
        
        # Petrosian Fractal Dimension (PFD)	pdf()	A scalar
        features.append(pyeeg.pfd(x_i))
        
        # Higuchi Fractal Dimension (HFD)	hfd()	A scalar
        features.append(pyeeg.hfd(x_i, K_MAX))
        
        # Hjorth mobility and complexity	hjorth()	Two scalars
        
        # Spectral Entropy (Shannon's entropy of RIRs)	spectral_entropy()	A scalar
        #features.append(pyeeg.spectral_entropy(x_i, FREQ_BANDS, SAMPLE_PER_SEC))
        
        # SVD Entropy	svd_entropy()	A scalar
        #features.append(pyeeg.svd_entropy(x_i, TAU, DE))
        
        # Fisher Information	fisher_info()	A scalar
        features.append(pyeeg.fisher_info(x_i, TAU, DE))
          
        # Detrended Fluctuation Analysis (DFA)	dfa()	A scalar
        features.append(pyeeg.dfa(x_i))
        
        # Hurst Exponent (Hurst)	hurst()	A scalar
        #features.append(pyeeg.hurst(x_i))
        
        # AX BASIC FEATUERS
        features.append(np.mean(x_i))
        features.append(np.std(x_i))
        features.append(np.min(x_i))
        features.append(np.max(x_i))
        features.append(np.sum(np.abs(x_i) > EPSILON))
        
        """
            DELTA_WAVE = [1, 4]
            THETA_WAVE = [4, 8]
            ALPHA_WAVE = [7.5, 12.5]
            BETA_WAVE = [13, 30]
        """
        delta = bandpower(x_i, DELTA_WAVE)
        theta = bandpower(x_i, THETA_WAVE)
        alpha = bandpower(x_i, ALPHA_WAVE)
        beta = bandpower(x_i, BETA_WAVE)
        total_energy = bandpower(x_i, TOTAL_ENERGY)
        
        
        features.append(delta)
        features.append(theta)
        features.append(alpha)
        features.append(beta)
        
        
        features.append(delta / total_energy)
        features.append(theta / total_energy)
        features.append(alpha / total_energy)
        features.append(beta / total_energy)
        
        
        
        all_featues.append(features)
    return np.array(all_featues)


def get_data_of_rat(X, y, i):
    sample_cnt = int(X.shape[0] / 3)
    if i == 0:
        return X[:sample_cnt, :], y[:sample_cnt]
    if i == 1:
        return X[sample_cnt: 2 * sample_cnt, :], y[sample_cnt: 2 * sample_cnt]
    if i == 2:
        return X[2 * sample_cnt:, :], y[2 * sample_cnt:]
    
    
def bandpower(data, band, window_sec=4, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.

    Examples
    ------
    1. Absolute and relative power in the delta band
        >>> delta = bandpower(data, 100, [0.5, 4])
        >>> delta_relative = bandpower(data, 100, [0.5, 4], relative=True)

    2. Delta / beta ratio
        >>> window_sec = 4
        >>> delta = bandpower(data, 100, [0.5, 4], window_sec)
        >>> beta = bandpower(data, 100, [12, 30], window_sec)
        >>> db_ratio = delta / beta
    """
    
    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if window_sec is not None:
        nperseg = window_sec * SAMPLE_PER_SEC
    else:
        nperseg = (2 / low) * SAMPLE_PER_SEC

    freqs, psd = welch(data, SAMPLE_PER_SEC, nperseg=nperseg, scaling='density')

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs < high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], freqs[idx_band])

    if relative:
        bp /= simps(psd, freqs)
    return bp

In [8]:
# Load the data
# train
train_feature_path = os.path.join(dir_path, "features/all_combined_train.csv")

# trest
test_feature_path = os.path.join(dir_path,"features/all_combined_test.csv")

# labels
train_target_path = os.path.join(dir_path, "features/all_combined_label.csv")

X_train = pd.read_csv(train_feature_path).values
y_train = pd.read_csv(train_target_path).values

X_test = pd.read_csv(test_feature_path).values
print(X_train.shape)
print(X_test.shape)

(64800, 49)
(43200, 49)


In [9]:
#Fit scaler on all data
X_total = np.concatenate((X_train, X_test))
print("X total shape -> ",X_total.shape)

scaler = StandardScaler().fit(X_total)

X total shape ->  (108000, 49)


In [30]:
# classifiers for CV

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
classifiers = [crf]
classifiers_names =["conditional random fields",  "RandomForestClassifier", "SVC"]

In [39]:
def sent2features(S):
    return dict(zip( range(S.shape[0]),  S.tolist() ) )

def sent2labels(s):
    if s[0] == 1:
        return ["1"]
    return ["0"]

# CROSS VALDATION
kfold = KFold(n_splits=3, shuffle=False, random_state=seed)

print("Start")
bmac_scores_rf = []
bmac_scores_svc = []
clf_scores_avg = []
clf_scores_std = []
for clf in classifiers:
    bmac_scores = []
    for train, valid in kfold.split(X_train):
        # get the folds
        X_train_fold = X_train[train]
        y_train_fold = y_train[train]

        X_valid_fold = X_train[valid]
        y_valid_fold = y_train[valid]

        X_train_fold_scaled = scaler.transform(X_train_fold)
        X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
        X_train_fold_scaled = [sent2features(X_train_fold_scaled[s, :]) for s in range(X_train_fold_scaled.shape[0])]
        y_train_fold = [sent2labels(y_train_fold[s, :]) for s in range(y_train_fold.shape[0])]

        X_valid_fold = [sent2features(X_valid_fold_scaled[s, :]) for s in range(X_valid_fold_scaled.shape[0])]
        y_valid_fold = [sent2labels(y_valid_fold[s, :]) for s in range(y_valid_fold.shape[0])]

        # fit classifier
        clf.fit(X_train_fold_scaled, y_train_fold)

        y_pred = clf.predict(X_valid_fold_scaled)
        

        bmac_score_rf = balanced_accuracy_score(y_valid_fold, y_pred)
        print(f"{len(bmac_scores)}: current balanced_accuracy_score: {bmac_score_rf}")

        bmac_scores.append(bmac_score_rf)
    
    clf_scores_avg.append(np.mean(bmac_scores))
    clf_scores_std.append(np.std(bmac_scores))

print("================================================================================")
for i in range(len(classifiers)):
    print(f"{classifiers_names[i]} BMAC avg score {clf_scores_avg[i]} +/- {clf_scores_std[i]}" )

Start


TypeError: object of type 'int' has no len()

In [42]:
bmac_scores = []
# CROSS VALDATION
kfold = KFold(n_splits=3, shuffle=False, random_state=seed)
for train, valid in kfold.split(X_train):
    # get the folds
    X_train_fold = X_train[train]
    y_train_fold = y_train[train]

    X_valid_fold = X_train[valid]
    y_valid_fold = y_train[valid]

    X_train_fold_scaled = scaler.transform(X_train_fold)
    X_valid_fold_scaled = scaler.transform(X_valid_fold)

    trainer = pycrfsuite.Trainer(verbose=True)
    trainer.set_params({
        'c1': 0.1,
        'c2': 0.01,  
        'max_iterations': 200,
        'feature.possible_transitions': False
    })


    # Submit training data to the trainer
    for i in range(X_train_fold_scaled.shape[0]):
        xseq = dict(
            zip(
                range(X_train_fold_scaled.shape[1]), 
                X_train_fold_scaled[i, :].tolist()
            )
        )
        #print(xseq)
        #print(y_train_fold[i, 0])
        trainer.append(xseq, "y")
                       
    trainer.train(f"crf_{len(bmac_scores)}.model")
    tagger = pycrfsuite.Tagger()
    tagger.open(f'crf_{len(bmac_scores)}.model')
    y_pred = [tagger.tag(X_valid_fold_scaled[i, :]) for i in range(X_valid_fold_scaled.shape[0])]


    bmac_score_rf = balanced_accuracy_score(y_valid_fold, y_pred)
    print(f"{len(bmac_scores)}: current balanced_accuracy_score: {bmac_score_rf}")

    bmac_scores.append(bmac_score_rf)
    

TypeError: object of type 'int' has no len()

In [30]:
#Scale, fit, predict
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc = SVC(class_weight="balanced")
svc.fit(X_train_scaled, y_train)
y_pred = svc.predict(X_test_scaled)
print(y_pred)

X total shape ->  (108000, 33)
[1 1 1 ... 1 1 1]


In [31]:
submission_name = "fv_allfeat.csv"

print(f"Class 1: {np.sum(y_pred == 1)}")
print(f"Class 2: {np.sum(y_pred == 2)}")
print(f"Class 3: {np.sum(y_pred == 3)}")

y_pred_df = pd.DataFrame(y_pred)
y_pred_df = y_pred_df.assign(Id=list(range(y_pred.shape[0])))
y_pred_df.columns = ['y', 'Id']
display(y_pred_df)


submission_folder = os.path.join(dir_path,"submissions/")
csv_file = submission_folder + submission_name

with open(csv_file, 'w') as csv:
    y_pred_df.to_csv(csv,index = False)
"""
Class 1: 23933
Class 2: 18553
Class 3: 714
"""

Class 1: 19527
Class 2: 21259
Class 3: 2414


,y,Id
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
5,1,5
6,1,6
7,1,7
8,1,8
9,1,9


'\nClass 1: 23933\nClass 2: 18553\nClass 3: 714\n'